In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import IPython
import IPython.display
import os

import numpy as np

import tensorflow as tf

import matplotlib.pyplot as plt
from profiling import cumTimer

In [ ]:
print(tf.__version__)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
tf.test.is_built_with_cuda()

# Import Data

In [ ]:
InputData = np.genfromtxt('Data/InputDataBenchmark.csv',delimiter=",")
t = np.genfromtxt('Data/timeBenchmark.csv',delimiter=",")
Data = np.genfromtxt('Data/OutputDataBenchmark.csv',delimiter=",")
print(f"OutData: %s, \nInputData: %s, \nTime: %s" %(np.shape(Data),np.shape(InputData),np.shape(t)))

# Split The Data

In [ ]:
from InputPipeline import DataPipeline

In [ ]:
pipe = DataPipeline(Data,InputData)
pipe.splitData(0.8,0.1,0.1)

In [ ]:
example_ind = 5
plt.plot(t,pipe.train_df[:,example_ind])
plt.grid()
plt.show()

# Data Windowing

In [ ]:
from InputPipeline import WindowGenerator

In [ ]:
OUT_STEPS = 30
multi_window = WindowGenerator(input_width=20,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=pipe.train_df,
                               val_df=pipe.val_df,
                               test_df=pipe.test_df,
                               md_train_df= pipe.meta_train_df,
                               md_test_df= pipe.meta_test_df,
                               md_val_df= pipe.meta_val_df,
                               batch_size = 500)

multi_window.plotexample()
multi_window

# Models

In [ ]:
from network import FeedBack, lstm_benchmark_model

## Normal

In [ ]:
model = lstm_benchmark_model(units = 64)
print(f"Input Shape: {multi_window.example[0].shape}")
print(f'Output Shape: {model(multi_window.example[0]).shape}')

In [ ]:
model.summary()

## Autoregressive Model

In [ ]:
ARmodel =  FeedBack(units=64, out_steps=OUT_STEPS)
print(f"Input Shape: {multi_window.example[0].shape}")
print(f'Output Shape: {ARmodel(multi_window.example[0]).shape}')

In [ ]:
ARmodel.summary()

## Load Weights

In [ ]:
models = os.listdir("Models/")
print(models)
print(f"Models/{models[0]}")

In [ ]:
ARmodel.load_weights(os.path.join("Models", models[0]))

# Plot Case

In [ ]:
%%time
predictions = multi_window.CalcCase(pipe.val_df[:,0],pipe.meta_val_df[:,0],t,ARmodel)

In [ ]:
example_ind = 2

In [ ]:
steps = 1
predictions = multi_window.plotCase(pipe.val_df[:,example_ind],
                                    timesteps = t[:multi_window.input_width+multi_window.label_width*steps],
                                    metadata= pipe.meta_val_df[:,example_ind],
                                    model= ARmodel,
                                    options={"showLines" : False,  "yLabel" :'x', "xLabel" :'Timesteps'})

In [ ]:
predictions = multi_window.plotCase(pipe.val_df[:,example_ind],timesteps = t,
                                    metadata = pipe.meta_val_df[:,example_ind],
                                    model=ARmodel)

# Create Animation

In [ ]:
from alive_progress import alive_bar

In [ ]:
from pltfigure import pltfigure
def animateDataset(dataset,metadataset,timesteps,name):
    predData = []
    tFinal = multi_window.input_width+multi_window.label_width*timesteps
    with alive_bar(dataset.shape[1]) as bar:
        for i in range(dataset.shape[1]):
            print("\nTest Case:{}".format(i))
            testCase = dataset[:,i]
            ts  = multi_window.CalcCase(testCase,metadataset[:,i],t[:tFinal],model)
            predData.append(ts)
            bar()
        predData =np.array(predData)
    pltfigure(dataset[:tFinal,:].T,predData,range(tFinal),"Data","Prediction",name)

In [ ]:
steps = 10
animateDataset(pipe.val_df,pipe.meta_val_df,steps,f"Graphs/Prediction@{steps}Steps.gif")